# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0036, -0.0921, -0.1289, -0.0605,  0.0055,  0.1755,  0.0633,  0.1758,
          0.0462, -0.1880],
        [ 0.0118, -0.1699, -0.1004, -0.1289, -0.1211,  0.0725, -0.0500,  0.2664,
          0.0377, -0.0897]], grad_fn=<AddmmBackward>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[ 0.3393, -0.2751,  0.0946,  0.3422, -0.0951, -0.0058, -0.2171, -0.3114,
         -0.0530,  0.1420],
        [ 0.0735, -0.2074,  0.0677,  0.2380, -0.1120, -0.0266, -0.0340, -0.2167,
         -0.0218,  0.1337]], grad_fn=<AddmmBackward>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0968,  0.0781,  0.0592,  0.0249, -0.4034,  0.1235,  0.0947, -0.2194,
          0.1622, -0.0741],
        [-0.0332,  0.0832, -0.0257, -0.0628, -0.2383,  0.0498,  0.2871, -0.1433,
          0.0734, -0.0920]], grad_fn=<AddmmBackward>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0026, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2860, grad_fn=<SumBackward0>)